In [143]:
import pandas as pd
import numpy as np
import MICE
import iMICE

In [144]:
df = pd.read_csv('titanic.csv')
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)
df = df.reset_index(drop = True)
df_1=df.copy()
df_1['AgeBinary'] = df['Age'].apply(lambda x: 1 if x >= 19 else 0)

actual = df_1['AgeBinary']
update = df_1.sample(int(0.1 * len(df))).index
df_1.at[update, 'AgeBinary'] = np.nan # changable

myFilter_train=df_1.AgeBinary.notnull()

y_train=actual[myFilter_train]
y_test=actual[update]

x_pclass=df['Pclass']
x_pclass_1=x_pclass[myFilter_train]
x_fare=df['Fare']
x_fare_1=x_fare[myFilter_train]
X_train=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})

x_pclass=df['Pclass']
x_pclass_1=x_pclass[update]
x_fare=df['Fare']
x_fare_1=x_fare[update]
X_test=pd.DataFrame({'Pclass':x_pclass_1.values, 'Fare':x_fare_1.values})


In [145]:
df_2 = df_1[['AgeBinary', 'Pclass', 'Fare']].copy()



In [146]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [147]:
df_3 = pd.DataFrame(mice_result, columns = ['Age Binary', 'Pclass', 'Fare'])
predict = df_3['Age Binary'].map(lambda x: int(x))

In [148]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(actual[update], predict[update]))
print(confusion_matrix(actual[update], predict[update])) 

0.8732394366197183
[[ 2  6]
 [ 3 60]]


# iMICE

In [149]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

In [150]:
df_3i = pd.DataFrame(imice_result, columns = ['Age Binary', 'Pclass', 'Fare'])
predicti = df_3i['Age Binary'].map(lambda x: int(x))

In [151]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(actual[update], predicti[update]))
print(confusion_matrix(actual[update], predicti[update])) 

0.8732394366197183
[[ 0  8]
 [ 1 62]]


# KNN

In [152]:
X_test=X_test.iloc[:,:].values
X_train=X_train.iloc[:,:].values
y_train=np.array(y_train)
y_test=np.array(y_test)

In [153]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

In [154]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7887323943661971
[[ 3  5]
 [10 53]]


# Random Forest

In [155]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

In [156]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8169014084507042
[[ 3  5]
 [ 8 55]]
